In this adversarial validation notebook we'll take a look at the difference between sites 1 and 2.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics

from sklearn import preprocessing
import gc


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
fnc_df = pd.read_csv("../input/trends-assessment-prediction/fnc.csv")
loading_df = pd.read_csv("../input/trends-assessment-prediction/loading.csv")

fnc_features, loading_features = list(fnc_df.columns[1:]), list(loading_df.columns[1:])
df = fnc_df.merge(loading_df, on="Id")

In [3]:
df.head()

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.368580,0.166876,0.438148,0.341007,-0.186251,0.049096,0.121417,-0.174268,-0.231578,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.151696,-0.024819,0.217504,0.418072,-0.227234,-0.064052,-0.143832,-0.118116,-0.054825,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,10003,0.343415,0.109974,0.741641,0.578558,-0.676446,-0.436960,-0.295663,-0.377790,-0.344963,...,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164
3,10004,0.132793,0.258255,0.490769,0.342717,0.091112,0.107969,0.029220,-0.026237,0.094742,...,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,10005,0.291921,0.251254,0.416470,0.511719,-0.362626,-0.164710,-0.289059,-0.015537,-0.087316,...,0.009702,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316


In [4]:
df.shape

(11754, 1405)

In [5]:
labels_df = pd.read_csv("../input/trends-assessment-prediction/train_scores.csv")
reveal_ID_site2 = pd.read_csv("../input/trends-assessment-prediction/reveal_ID_site2.csv")
labels_df.head()

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,66.532630,NaN,NaN,52.108977,69.993075
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421


In [6]:
labels_df.shape

(5877, 6)

In [7]:
reveal_ID_site2.head()

,Id
0,10012
1,10014
2,10020
3,10034
4,10059


In [8]:
reveal_ID_site2.shape

(510, 1)

In [9]:
labels_df.shape

(5877, 6)

In [10]:
df['target'] = np.nan

In [11]:
df.isna().sum().sum()

11754

In [12]:
df.loc[df.Id.isin(labels_df.Id), 'target'] = 0
df.loc[df.Id.isin(reveal_ID_site2.Id), 'target'] = 1
df.dropna(inplace=True)

df.shape

(6387, 1406)

In [13]:
to_drop=[
         
         'IC_05',
         'IC_16',
         'IC_10',
    
        ]
to_drop_67=['IC_20',
 'IC_18',
 'IC_11',
 'IC_28',
 'IC_05',
 'CBN(4)_vs_CON(37)',
 'IC_16',
 'IC_21',
 'IC_13',
 'IC_24',
 'CON(83)_vs_CON(48)',
 'CON(83)_vs_CON(33)',
 'DMN(17)_vs_CON(33)',
 'CON(79)_vs_SMN(54)',
 'IC_07',
 'CON(83)_vs_CON(67)',
 'IC_04',
 'CON(55)_vs_CON(33)',
 'IC_15',
 'DMN(71)_vs_CON(43)',
 'SCN(99)_vs_SCN(98)',
 'IC_02']
# 0.6885 with IC18 
# 0.6855 with IC18 and IC15
# 0.694727 without 'CBN(4)_vs_CON(38)'
# 0.696675 without 'SCN(99)_vs_SCN(98)'
# 0.697862    with 'SCN(99)_vs_SCN(98)'

In [14]:
to_drop_67=['IC_20',
         'IC_02',
         'IC_05',
         'IC_16',
         'IC_10',
         'IC_08',
         'CBN(4)_vs_CON(37)',
         'CBN(4)_vs_CON(38)',
         'SCN(99)_vs_SCN(98)',
         'DMN(23)_vs_CON(37)',
         'DMN(40)_vs_CON(48)',
         'DMN(17)_vs_DMN(40)',
         'DMN(17)_vs_CON(88)',
         'DMN(17)_vs_CON(33)',
         'CON(79)_vs_SMN(54)',
         'CON(55)_vs_SCN(45)',
         'CON(88)_vs_SMN(54)',
         'CON(83)_vs_CON(48)',
         'CON(83)_vs_CON(67)',
         'CON(83)_vs_CON(37)',
         'CON(83)_vs_CON(33)',
        ]
len(to_drop)

3

In [ ]:
#TRANSFORMATIONS
df[to_drop]=df[to_drop].pow(2)
df[fnc_features]=df[fnc_features].mul(1/600)

In [ ]:
features = loading_features + fnc_features
features=list(set(features)-set(['Id','target','IC_20','IC_18','CBN(4)_vs_CON(37)','IC_02']))

In [ ]:
# #-------Normalizing------------------------
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df[features]= scaler.fit_transform(df[features])
# #----------------------------------------------------

In [ ]:
train = df[features].values
target = df['target'].values
train, test, y_train, y_test = model_selection.train_test_split(train, target, test_size=0.33, random_state=42, shuffle=True)
del target
gc.collect()

In [ ]:
train = lgb.Dataset(train, label=y_train)
test = lgb.Dataset(test, label=y_test)

In [ ]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.05,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         'tree_learner':'feature_parallel',
         "bagging_fraction": 0.9 ,
         "bagging_seed": 56,
         "metric": 'auc',
         "verbosity": -1}

In [ ]:
num_round = 10000
clf = lgb.train(param, train, num_round, valid_sets = [train, test], verbose_eval=50, early_stopping_rounds = 100)

The AUC of 0.88 is much higher than the AUC of 0.72 for the "regular" train-test adversarial validation. Seems taht we really ahve to be very careful about the differences between sites 1 and 2.

Let us now take a look at the most important features 

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(),features)), columns=['Value','Feature'])

plt.figure(figsize=(20, 20))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).head(100))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

Here it would appear that the worst "culprits" are the IC features. Let's again list the top 20 most important ones, at least according to this measure.

In [ ]:
list(feature_imp.sort_values(by="Value", ascending=False)['Feature'].values)[:21]

In [ ]:
['IC_20',
 'IC_18',
 'IC_05',
 'CBN(4)_vs_CON(37)',
 'IC_11',
 'IC_28',
 'IC_16',
 'IC_21',
 'CON(83)_vs_CON(48)',
 'IC_24',
 'CON(83)_vs_CON(33)',
 'IC_13',
 'CON(83)_vs_CON(67)',
 'DMN(17)_vs_CON(33)',
 'CON(83)_vs_CON(37)',
 'IC_30',
 'CON(79)_vs_SMN(54)',
 'CON(55)_vs_SCN(45)',
 'DMN(23)_vs_CON(37)',
 'IC_07',
 'SCN(99)_vs_SCN(98)']

In [ ]:
feature_imp.head()